# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix (q=0,ω=0) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.234965889697                   -0.50    8.0
  2   -7.250338852319       -1.81       -1.40    1.0
  3   -7.251300694551       -3.02       -2.13    3.0
  4   -7.251263932438   +   -4.43       -2.19    4.0
  5   -7.251328449105       -4.19       -2.66    2.0
  6   -7.251337581758       -5.04       -3.08    2.0
  7   -7.251338752149       -5.93       -3.86    1.0
  8   -7.251338791995       -7.40       -4.11    4.0
  9   -7.251338797446       -8.26       -4.53    1.0
 10   -7.251338798681       -8.91       -5.30    3.0
 11   -7.251338798697      -10.80       -5.55    3.0
 12   -7.251338798702      -11.34       -5.80    1.0
 13   -7.251338798704      -11.61       -6.30    2.0
 14   -7.251338798705      -12.59       -6.97    3.0
 15   -7.251338798705      -13.85       -7.47    3.0
 16   -7.251338798705   +  -14.75       -7.76    3.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Apply ε† = 1 - χ0 (vc + fxc)

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end

eps_fun (generic function with 1 method)

eager diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.06124735466708176
[ Info: Arnoldi iteration step 2: normres = 0.37009042507986056
[ Info: Arnoldi iteration step 3: normres = 1.0240381046000449
[ Info: Arnoldi iteration step 4: normres = 0.3162619624783024
[ Info: Arnoldi iteration step 5: normres = 0.5488469637558848
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (6.62e-02, 6.47e-02, 4.85e-01, 2.39e-01, 1.84e-02)
[ Info: Arnoldi iteration step 6: normres = 0.27650426476546525
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.61e-02, 1.89e-01, 1.46e-01, 1.15e-01, 6.24e-02)
[ Info: Arnoldi iteration step 7: normres = 0.09652719667095032
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (7.42e-04, 1.65e-02, 8.63e-03, 4.85e-02, 6.94e-02)
[ Info: Arnoldi iteration step 8: normres = 0.1214238345709835
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (3.94e-